In [11]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from animalshelter import AnimalShelter
import base64
username = "noahkhomer"
password = "noahkhomerpassword"
db = AnimalShelter()
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
app = JupyterDash(__name__)
image_filename = "/home/noahkhomer_snhu/Grazioso Salvare Logo (1).png"
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Noah Khomer'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    html.H2('Interactive Data Table and Geolocation Chart'),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",
        style_table={'overflowX': 'auto'},
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    else:  # Reset
        query = {}

    filtered_data = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)
    return filtered_data.to_dict('records')
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Breed Distribution')
            )
        ]
    return "No data to display."
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        return "No data available for mapping."

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0] if selected_rows else 0

    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                              children=[
                                  dl.Tooltip(dff.iloc[row]['breed']),
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row]['name'])
                                  ])
                              ])
                ])
        ]
    return "Geolocation data not available."
app.run_server(debug=True)


Dash app running on http://127.0.0.1:13782/
